In [34]:
import pandas as pd

# Reading the file csv
df_1 = pd.read_csv(f"tennis_data/away_team.csv")
df_2 = pd.read_csv(f"tennis_data/home_team.csv")

# Concatenate two csv files
merge_data = pd.concat([df_1,df_2], ignore_index=True)

# Filtering the available data based on needs and removing duplicate data and nan data in the height column
final_data = merge_data[["match_id", "name", "height"]].drop_duplicates(subset='name').dropna(subset=['height'])

# Calculate the mean height of the players
mean_height = final_data['height'].mean()
'The mean height of the players is '+ str(mean_height)

'The mean height of the players is 1.8213657056145676'